# Séance 6

## Objectifs de la séance

- Résolution graphique de la détermination du correcteur PD par placement fréquentiel
- Résolution graphique de la correction du correcteur PID

In [1]:
from IPython.display import Image, display, Markdown

from control.matlab import *  # Python Control Systems Toolbox (compatibility with MATLAB)
import numpy as np              # Library to manipulate array and matrix
import matplotlib.pyplot as plt # Library to create figures and plots
import math # Library to be able to do some mathematical operations
import ReguLabFct as rlf # Library useful for the laboratory of regulation of HELMo Gramme

## Régulateur PD

### Exercice 6 page 7-15 (fait au cours)

## Régulateur PID

Soit le système asservi à retour unitaire dont la fonction de boucle ouverte est :

$$
 G(p)=\frac{20}{(5+p)(2p+3)(4+3p)}
$$

On demande de déterminer le correcteur dont le but est de permettre une erreur statique nulle et une erreur de
vitesse minimale, tout en assurant une réponse indicielle dont le dépassement est limité à 5%.

### Exercice 7 page 7-18 (résolution dans les notes) (facultatif)

## Exercices récapitulatifs

### Exercice type examen (cf. HELMo Learn)

### Application 5 (cf. p. "application-8")

Effectuez la résolution par programmation Python.

### Exercice 3 page 7-27

### Application 3  (cf. p. "application-4")

Effectuez la résolution par programmation Python.